<a href="https://colab.research.google.com/github/samsoneul/Carbon-Emissions-Analyzer/blob/main/carbon%20emmissions%20analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
import pandas as pd
import numpy as np
import os


print(os.getcwd())

#load data

file_path = '/content/drive/MyDrive/PDGES-GHGRP-GHGEmissionsGES-2004-Present (1).csv'
data = pd.read_csv(file_path, encoding='ISO-8859-1')


/content


In [41]:
#explore the data
# print(data.head())

print(data.isnull().sum())


GHGRP ID / Numéro d'identification de PDGES                                                           0
Reference Year / Année de référence                                                                   0
Facility Name / Nom de l'installation                                                                 0
Facility Location / Emplacement de l'installation                                                  3323
Facility City or District or Municipality / Ville ou District ou Municipalité de l'installation    1252
                                                                                                   ... 
C6F14 (tonnes CO2e / tonnes éq. CO2)                                                                  0
PFC Total (tonnes CO2e / tonnes éq. CO2)                                                              1
SF6 (tonnes)                                                                                       9660
SF6 (tonnes CO2e / tonnes éq. CO2)                              

In [63]:
#select columns used for analysis
key_columns = [
    'GHGRP ID / Numéro d\'identification de PDGES',  # Facility identifier
    'Facility Name / Nom de l\'installation',
    'Facility Province or Territory / Province ou territoire de l\'installation',

]
#add coulmns that have CO2 emmision data
for col in data.columns.to_list():
  if 'tonnes CO2e' in col:
    key_columns.append(col)
print(key_columns)

["GHGRP ID / Numéro d'identification de PDGES", "Facility Name / Nom de l'installation", "Facility Province or Territory / Province ou territoire de l'installation", 'CO2 (tonnes CO2e / tonnes éq. CO2)', 'CH4 (tonnes CO2e / tonnes éq. CO2)', 'N2O (tonnes CO2e / tonnes éq. CO2)', ' HFC-23 (tonnes CO2e / tonnes éq. CO2)', 'HFC-32 (tonnes CO2e / tonnes éq. CO2)', 'HFC-125 (tonnes CO2e / tonnes éq. CO2)', 'HFC-134a (tonnes CO2e / tonnes éq. CO2)', 'HFC-143a (tonnes CO2e / tonnes éq. CO2)', 'HFC-152a (tonnes CO2e / tonnes éq. CO2)', 'HFC-41 (tonnes CO2e / tonnes éq. CO2)', 'HFC-43-10mee (tonnes CO2e / tonnes éq. CO2)', 'HFC-134 (tonnes CO2e / tonnes éq. CO2)', 'HFC-143 (tonnes CO2e / tonnes éq. CO2)', 'HFC-227ea (tonnes CO2e / tonnes éq. CO2)', 'HFC-236fa (tonnes CO2e / tonnes éq. CO2)', 'HFC-245ca (tonnes CO2e / tonnes éq. CO2)', 'HFC Total (tonnes CO2e / tonnes éq. CO2)', 'CF4 (tonnes CO2e / tonnes éq. CO2)', 'C2F6 (tonnes CO2e / tonnes éq. CO2)', 'C3F8 (tonnes CO2e / tonnes éq. CO2)', 'C

In [64]:



# Identify columns with missing values
cols_with_missing = data[key_columns].isnull().any()
cols_with_missing=cols_with_missing.index.tolist()
print(cols_with_missing)

# Create a report dictionary to track imputations
imputation_report = {}

# Impute missing values with the median for numerical columns
# Identify columns with missing values


# Create a report dictionary to track imputations
imputation_report = {}

for col in cols_with_missing:
    # Initialize report entry
    imputation_report[col] = {
        'n_missing': data[col].isnull().sum(),
        'method': None,
        'imputed_value': None
    }

    # Handle numerical columns
    if pd.api.types.is_numeric_dtype(data[col]):
        if 'tonnes CO2e' in col or 'tonnes' in col:
            # For emissions data, 0 often means no emissions
            data[col].fillna(0, inplace=True)  # Changed from string '0' to numeric 0
            imputation_report[col].update({
                'method': 'zero (emissions assumption)',
                'imputed_value': 0
            })
        else:
            # Use median for other numericals
            median_val = data[col].median()
            data[col].fillna(median_val, inplace=True)
            imputation_report[col].update({
                'method': 'median',
                'imputed_value': median_val
            })

    # Handle categorical columns
    else:
        if 'GHGRP ID' in col or 'Numéro d\'identification' in col:
            # Special handling for IDs
            data[col].fillna('MISSING_ID', inplace=True)
            imputation_report[col].update({
                'method': 'missing_id placeholder',
                'imputed_value': 'MISSING_ID'
            })
        elif 'Facility' in col or 'Emplacement' in col:
            # Facility-related info
            data[col].fillna('UNKNOWN_FACILITY', inplace=True)
            imputation_report[col].update({
                'method': 'unknown_facility placeholder',
                'imputed_value': 'UNKNOWN_FACILITY'
            })
        else:
            # General categorical columns
            mode_stats = data[col].value_counts(normalize=True)
            if mode_stats.iloc[0] > 0.6:  # Strong mode exists
                mode_val = mode_stats.index[0]
                data[col].fillna(mode_val, inplace=True)
                imputation_report[col].update({
                    'method': 'mode (dominant category)',
                    'imputed_value': mode_val,
                    'mode_ratio': float(mode_stats.iloc[0])  # Store dominance ratio
                })
            else:
                # For weak/no dominant mode
                if not pd.api.types.is_categorical_dtype(data[col]):
                    data[col] = data[col].astype('category')
                data[col] = data[col].cat.add_categories('MULTIPLE_COMMON_VALUES')
                data[col].fillna('MULTIPLE_COMMON_VALUES', inplace=True)
                imputation_report[col].update({
                    'method': 'multiple_common_values flag',
                    'imputed_value': 'MULTIPLE_COMMON_VALUES',
                    'top_category': mode_stats.index[0],
                    'top_ratio': float(mode_stats.iloc[0])
                })

# Print results
print("\nMissing values after cleaning:")
print(data.isnull().sum())

# Show imputation report
print("\nImputation Methods Used:")
for col, report in imputation_report.items():
    print(f"\n{col}:")
    for k, v in report.items():
        print(f"  {k}: {v}")

["GHGRP ID / Numéro d'identification de PDGES", "Facility Name / Nom de l'installation", "Facility Province or Territory / Province ou territoire de l'installation", 'CO2 (tonnes CO2e / tonnes éq. CO2)', 'CH4 (tonnes CO2e / tonnes éq. CO2)', 'N2O (tonnes CO2e / tonnes éq. CO2)', ' HFC-23 (tonnes CO2e / tonnes éq. CO2)', 'HFC-32 (tonnes CO2e / tonnes éq. CO2)', 'HFC-125 (tonnes CO2e / tonnes éq. CO2)', 'HFC-134a (tonnes CO2e / tonnes éq. CO2)', 'HFC-143a (tonnes CO2e / tonnes éq. CO2)', 'HFC-152a (tonnes CO2e / tonnes éq. CO2)', 'HFC-41 (tonnes CO2e / tonnes éq. CO2)', 'HFC-43-10mee (tonnes CO2e / tonnes éq. CO2)', 'HFC-134 (tonnes CO2e / tonnes éq. CO2)', 'HFC-143 (tonnes CO2e / tonnes éq. CO2)', 'HFC-227ea (tonnes CO2e / tonnes éq. CO2)', 'HFC-236fa (tonnes CO2e / tonnes éq. CO2)', 'HFC-245ca (tonnes CO2e / tonnes éq. CO2)', 'HFC Total (tonnes CO2e / tonnes éq. CO2)', 'CF4 (tonnes CO2e / tonnes éq. CO2)', 'C2F6 (tonnes CO2e / tonnes éq. CO2)', 'C3F8 (tonnes CO2e / tonnes éq. CO2)', 'C

<ipython-input-64-39b355960ec7>:46: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].fillna('MISSING_ID', inplace=True)
<ipython-input-64-39b355960ec7>:53: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 

In [47]:
for col in data.columns.to_list():
  if 'tonnes CO2e' in col:
    key_columns.append(col)
print(key_columns)

["GHGRP ID / Numéro d'identification de PDGES", "Facility Name / Nom de l'installation", "Facility Province or Territory / Province ou territoire de l'installation", 'Latitude', 'Longitude', 'Total Emissions (tonnes CO2e) / Émissions totales (tonnes éq. CO2)', 'CO2 (tonnes)', 'CH4 (tonnes CO2e / tonnes éq. CO2)', 'CO2 (tonnes CO2e / tonnes éq. CO2)', 'CH4 (tonnes CO2e / tonnes éq. CO2)', 'N2O (tonnes CO2e / tonnes éq. CO2)', ' HFC-23 (tonnes CO2e / tonnes éq. CO2)', 'HFC-32 (tonnes CO2e / tonnes éq. CO2)', 'HFC-125 (tonnes CO2e / tonnes éq. CO2)', 'HFC-134a (tonnes CO2e / tonnes éq. CO2)', 'HFC-143a (tonnes CO2e / tonnes éq. CO2)', 'HFC-152a (tonnes CO2e / tonnes éq. CO2)', 'HFC-41 (tonnes CO2e / tonnes éq. CO2)', 'HFC-43-10mee (tonnes CO2e / tonnes éq. CO2)', 'HFC-134 (tonnes CO2e / tonnes éq. CO2)', 'HFC-143 (tonnes CO2e / tonnes éq. CO2)', 'HFC-227ea (tonnes CO2e / tonnes éq. CO2)', 'HFC-236fa (tonnes CO2e / tonnes éq. CO2)', 'HFC-245ca (tonnes CO2e / tonnes éq. CO2)', 'HFC Total (t

In [ ]:
print(data.head())